In [7]:
!pip install diffusers
!pip install transformers
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git


%cd stylegan2-ada-pytorch


import sys
sys.path.append('.')

fatal: destination path 'stylegan2-ada-pytorch' already exists and is not an empty directory.
/content/stylegan2-ada-pytorch


In [8]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
sd_pipeline = StableDiffusionPipeline.from_pretrained(model_id,
                                                      torch_dtype=torch.float16)
sd_pipeline = sd_pipeline.to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
def get_completion_sd(prompt):
  negative_prompt = """
  simple background, duplicate, low quality, lowest quality,
  bad anatomy, bad proportions, extra digits, lowres, username,
  artist name, error, duplicate, watermark, signature, text,
  extra digit, fewer digits, worst quality, jpeg artifacts, blurry
  """
  return sd_pipeline(prompt, negative_prompt=negative_prompt).images[0]

In [10]:
prompt = """baby llama, wearing red muffler,
grazing, open field, sunset
"""
print(prompt)
sd_image = get_completion_sd(prompt)
sd_image.save("./llama.jpg")

baby llama, wearing red muffler,
grazing, open field, sunset



  0%|          | 0/50 [00:00<?, ?it/s]

### Build AI Image Generator

In [1]:
!pip install gradio==3.48.0

In [2]:
import gradio as gr
from PIL import Image
import numpy as np
from torchvision import transforms


In [3]:
def get_completion(input_type,category, style, color_scheme, pattern, material, fit, occasion, season,custom_prompt):
     if input_type == "Dropdowns":
        prompt = f"A professional fashion model wearing a {category}, designed in {style} style with {color_scheme} colors and a {pattern} pattern. The outfit is crafted from {material}, featuring a {fit} fit, perfect for {occasion} and ideal for {season} wear. The model has a clear, sharp, defined face and a neutral expression, ensuring a fashion-focused presentation. Captured in a high-end fashion studio with professional lighting, this ultra-realistic 8K UHD image highlights the outfit’s intricate textures and design. The model poses confidently against a pure white background, maintaining a clean and professional aesthetic."
     else:
        prompt = custom_prompt


     negative_prompt = """
    simple background, duplicate, low quality, lowest quality,
    bad anatomy, bad proportions, extra digits, lowres, username,
    artist name, error, duplicate, watermark, signature, text,
    extra digit, fewer digits, worst quality, jpeg artifacts, blurry face,
    navel
    """

     return sd_pipeline(prompt, negative_prompt=negative_prompt).images[0]


In [11]:
import dnnlib, legacy
stylegan2_network_pkl = "/content/drive/My Drive/stylegan2-ffhq-config-e.pkl"  # e.g., a local file or URL
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with dnnlib.util.open_url(stylegan2_network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
model_path = "/content/drive/My Drive/stylegan2-ffhq-config-e.pkl"

In [13]:
def encode_image(image):
    # Dummy encoder: returns a random latent vector of the correct size.
    # Replace this with an actual inversion step.
    return torch.randn([1, G.z_dim]).to(device)

In [14]:
def refine_with_stylegan2(sd_image):
    """
    Refine the Stable Diffusion output using the pretrained StyleGAN2 generator.
    This function inverts the image (dummy inversion here) and then synthesizes a refined image.
    """
    # Ensure image is in RGBA format
    if sd_image.mode != "RGBA":
        sd_image = sd_image.convert("RGBA")
    with torch.no_grad():
        refined_tensor = G.synthesis(latent, noise_mode="const")

    # Post-process: convert tensor from [-1,1] to [0,255]
    refined_tensor = (refined_tensor.clamp(-1, 1) + 1) / 2 * 255
    refined_tensor = refined_tensor[0].permute(1, 2, 0).cpu().numpy().astype(np.uint8)
    return Image.fromarray(refined_tensor)

In [15]:
categories = ["Dress", "T-shirt", "Jeans", "Jacket","Shirt and Jeans","Lehenga","Churidar","Saree","Gown"]
styles = ["Modern", "Vintage", "Casual", "Elegant","Traditional"]
color_schemes = ["Monochrome", "Pastel", "Vibrant"]
patterns = ["Striped", "Floral", "Plain","Embroidery"]
materials = ["Cotton", "Silk", "Denim", "Leather","Georgette","Crepe"]
fits = ["Slim", "Regular", "Loose"]
occasions = ["Formal", "Casual", "Party"]
seasons = ["Summer", "Winter","Normal"]

def show_generator():
    return {welcome_page: gr.update(visible=False), generator_page: gr.update(visible=True)}

# Handle feedback submission
def submit_user_feedback(rating, text):
    return f"Thank you for your feedback! You rated us {rating}/5."

# HTML for start screen
welcome_html = """
<!DOCTYPE html>
<html>
<head>
<style>
  body, html {
    margin: 0;
    padding: 0;
    width: 100%;
    height: 100%;
    font-family: 'Arial', sans-serif;
  }

  .scene {
    width: 100%;
    height: 100vh;
    background: linear-gradient(135deg, #000428, #004e92);
    display: flex;
    justify-content: center;
    align-items: center;
  }

  .card-container {
    width: 60%;
    height: 60%;
    background: orange;
    border-radius: 20px;
    display: flex;
    align-items: center;
    justify-content: center;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.5);
    text-align: center;
  }

  .title {
    font-size: 3em;
    font-weight: bold;
    color: white; /* Changed from orange to white */
  }

  .subtitle {
    font-size: 1.5em;
    color: white; /* Changed from orange to white */
    margin-bottom: 20px;
  }

  .start-button {
    padding: 15px 40px;
    background: linear-gradient(135deg, #FFA500, #FF7300); /* Orange gradient */
    color: white; /* Text color */
    font-size: 1.9em;
    border: none;
    border-radius: 50px;
    cursor: pointer;
    box-shadow: 0 5px 15px rgba(255, 165, 0, 0.3);
    transition: transform 0.2s ease-in-out;
  }

  .start-button:hover {
    transform: scale(1.05);
  }
</style>
</head>
<body>
  <div class="scene">
    <div class="card-container">
      <div>
        <h1 class="title">Fashion AI Generator</h1>
        <p class="subtitle">Create stunning fashion designs with AI!</p>
        <br><br><br><br>
        <button class="start-button" onclick="document.getElementById('startButtonGradio').click();">START</button>
      </div>
    </div>
  </div>
</body>
</html>

"""

# Build Gradio UI
with gr.Blocks() as demo:
    # Start Page
    with gr.Group(visible=True) as welcome_page:
        gr.HTML(welcome_html)
        start_btn = gr.Button("Start", visible=False, elem_id="startButtonGradio")

    # Fashion Generator Page (Scrollable)
    with gr.Group(visible=False) as generator_page:
        gr.Markdown("## Fashion Design Generator")

        input_type = gr.Radio(choices=["Dropdowns", "Manual Text"], label="Select Input Type", value="Dropdowns")
        with gr.Row():
            with gr.Column(scale=1):
                category = gr.Dropdown(choices=categories, label="Category", value="Dress")
                style = gr.Dropdown(choices=styles, label="Style", value="Modern")
                color_scheme = gr.Dropdown(choices=color_schemes, label="Color Scheme", value="Vibrant")
                pattern = gr.Dropdown(choices=patterns, label="Pattern", value="Plain")
                material = gr.Dropdown(choices=materials, label="Material", value="Cotton")
                fit = gr.Dropdown(choices=fits, label="Fit", value="Regular")
                occasion = gr.Dropdown(choices=occasions, label="Occasion", value="Casual")
                season = gr.Dropdown(choices=seasons, label="Season", value="Summer")

                custom_prompt = gr.Textbox(label="Custom Description", placeholder="Describe your fashion design...", interactive=True)
                generate_btn = gr.Button("Generate Fashion Image", variant="primary")

            with gr.Column(scale=1):
                output_image = gr.Image(label="Generated Fashion Design", type="pil")

        # Feedback Section (Scroll Enabled)
        with gr.Group():
            gr.Markdown("### We'd Love Your Feedback!")
            rating = gr.Slider(minimum=1, maximum=5, step=1, label="Rate (1-5)", value=5)
            feedback_text = gr.Textbox(label="Your Feedback", lines=3, placeholder="Your thoughts on the generated design")
            submit_feedback = gr.Button("Submit Feedback")
            feedback_message = gr.Markdown(visible=True)

    # Event Handlers
    start_btn.click(show_generator, outputs=[welcome_page, generator_page])
    generate_btn.click(get_completion, inputs=[input_type,category, style, color_scheme, pattern, material, fit, occasion, season,custom_prompt], outputs=output_image)
    submit_feedback.click(submit_user_feedback, inputs=[rating, feedback_text], outputs=feedback_message)


# Launch Gradio App
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.48.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://b05da860a80b2ebff7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [17]:
import torch
import numpy as np
import cv2
import gradio as gr
from PIL import Image
import io
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")

# Load models
try:
    from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline

    # Text-to-image pipeline for Fashion Generation with lower precision for memory efficiency
    txt2img_pipeline = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)

    # Enable attention slicing for lower memory usage
    txt2img_pipeline.enable_attention_slicing()

    # Image-to-image pipeline for Similar Image Generation
    img2img_pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)

    img2img_pipeline.enable_attention_slicing()

    MODELS_LOADED = True
    logger.info("Models loaded successfully")

except Exception as e:
    logger.error(f"Error loading models: {e}")
    MODELS_LOADED = False
    # Create dummy pipelines for demo purposes if loading fails
    class DummyPipeline:
        def __call__(self, **kwargs):
            class DummyOutput:
                images = [Image.new('RGB', (512, 512), color=(200, 200, 200))]
            return DummyOutput()
    txt2img_pipeline = DummyPipeline()
    img2img_pipeline = DummyPipeline()

# Define categories and attributes for Fashion Generator
categories = ["Dress", "T-shirt", "Jeans", "Jacket", "Shirt and Jeans", "Lehenga", "Churidar", "Saree", "Gown"]
styles = ["Modern", "Vintage", "Casual", "Elegant", "Traditional"]
color_schemes = ["Monochrome", "Pastel", "Vibrant"]
patterns = ["Striped", "Floral", "Plain", "Embroidery"]
materials = ["Cotton", "Silk", "Denim", "Leather", "Georgette", "Crepe"]
fits = ["Slim", "Regular", "Loose"]
occasions = ["Formal", "Casual", "Party"]
seasons = ["Summer", "Winter", "Normal"]

# Function for Fashion Generation
def get_completion(input_type, category, style, color_scheme, pattern, material, fit, occasion, season, custom_prompt):
    """Generate fashion image based on selected parameters or custom prompt"""
    try:
        if not MODELS_LOADED:
            return Image.new('RGB', (512, 512), color=(200, 200, 200))

        if input_type == "Dropdowns":
            prompt = f"High quality fashion photography of a {fit} {style} {category} made of {material} with {pattern} pattern in {color_scheme} colors, suitable for {occasion} wear during {season} season"
        else:
            prompt = custom_prompt if custom_prompt else "A fashion item"

        logger.info(f"Generating image with prompt: {prompt}")

        # Use lower number of inference steps for faster generation
        image = txt2img_pipeline(
            prompt=prompt,
            num_inference_steps=25,
            guidance_scale=7.5
        ).images[0]

        return image
    except Exception as e:
        logger.error(f"Error in fashion generation: {e}")
        # Return a red image with error message
        error_img = Image.new('RGB', (512, 512), color=(255, 200, 200))
        return error_img

# Function for Similar Image Generation
def generate_similar_image(reference_image, strength=0.7):
    """Generate an image similar to the uploaded reference image using img2img"""
    try:
        if reference_image is None or not MODELS_LOADED:
            return Image.new('RGB', (512, 512), color=(200, 200, 200))

        if isinstance(reference_image, np.ndarray):
            reference_image = Image.fromarray(reference_image)

        # Ensure image is RGB
        reference_image = reference_image.convert("RGB").resize((512, 512))

        logger.info(f"Generating similar image with strength: {strength}")

        similar_image = img2img_pipeline(
            prompt="high quality fashion item",
            image=reference_image,
            strength=strength,
            guidance_scale=7.5,
            num_inference_steps=25
        ).images[0]

        return similar_image
    except Exception as e:
        logger.error(f"Error in similar image generation: {e}")
        # Return image with error
        error_img = Image.new('RGB', (512, 512), color=(255, 200, 200))
        return error_img

# Function to overlay face onto model (unchanged)
def overlay_face(base_image, face_image):
    """Overlay the face image onto the base image (placeholder implementation)"""
    try:
        if face_image is None or base_image is None:
            return base_image

        # Convert images to numpy arrays if they're not already
        if isinstance(base_image, Image.Image):
            base_np = np.array(base_image)
        else:
            base_np = base_image

        if isinstance(face_image, Image.Image):
            face_np = np.array(face_image)
        else:
            face_np = face_image

        # Convert to RGB if needed
        if len(base_np.shape) == 2 or base_np.shape[2] == 1:
            base_np = cv2.cvtColor(base_np, cv2.COLOR_GRAY2RGB)
        if len(face_np.shape) == 2 or face_np.shape[2] == 1:
            face_np = cv2.cvtColor(face_np, cv2.COLOR_GRAY2RGB)

        # Detect face in the face image
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        gray_face = cv2.cvtColor(face_np, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray_face, 1.1, 4)

        if len(faces) == 0:
            return Image.fromarray(base_np)

        (fx, fy, fw, fh) = faces[0]
        extracted_face = face_np[fy:fy+fh, fx:fx+fw]

        # Detect face in base image
        gray_base = cv2.cvtColor(base_np, cv2.COLOR_RGB2GRAY)
        base_faces = face_cascade.detectMultiScale(gray_base, 1.1, 4)

        if len(base_faces) == 0:
            # If no face detected in base, place at a reasonable position
            bx, by = base_np.shape[1] // 2 - fw // 2, base_np.shape[0] // 4 - fh // 2
        else:
            (bx, by, bw, bh) = base_faces[0]
            # Resize extracted face to match base face size
            extracted_face = cv2.resize(extracted_face, (bw, bh))
            fw, fh = bw, bh

        # Create mask for blending
        mask = np.zeros((fh, fw), dtype=np.uint8)
        mask = cv2.ellipse(mask, (fw//2, fh//2), (fw//2-5, fh//2-5), 0, 0, 360, 255, -1)
        mask = cv2.GaussianBlur(mask, (11, 11), 0)

        # Apply mask and blend faces
        for c in range(3):
            base_np[by:by+fh, bx:bx+fw, c] = (
                base_np[by:by+fh, bx:bx+fw, c] * (255 - mask) / 255 +
                extracted_face[:, :, c] * mask / 255
            )

        return Image.fromarray(base_np)
    except Exception as e:
        logger.error(f"Error in face overlay: {e}")
        return base_image if isinstance(base_image, Image.Image) else Image.fromarray(base_image)

# Function for Virtual Try-On (unchanged)
def virtual_try_on(user_image, cloth_image):
    """Overlay cloth image onto user image"""
    try:
        if user_image is None or cloth_image is None:
            return None

        # Convert to numpy arrays if needed
        if isinstance(user_image, Image.Image):
            user_img = np.array(user_image)
        else:
            user_img = user_image

        if isinstance(cloth_image, Image.Image):
            cloth_img = np.array(cloth_image)
        else:
            cloth_img = cloth_image

        # Convert to RGB if needed
        if len(user_img.shape) == 2 or user_img.shape[2] == 1:
            user_img = cv2.cvtColor(user_img, cv2.COLOR_GRAY2RGB)
        if len(cloth_img.shape) == 2 or cloth_img.shape[2] == 1:
            cloth_img = cv2.cvtColor(cloth_img, cv2.COLOR_GRAY2RGB)

        # Extract dress from background
        gray_cloth = cv2.cvtColor(cloth_img, cv2.COLOR_RGB2GRAY)
        _, mask = cv2.threshold(gray_cloth, 200, 255, cv2.THRESH_BINARY_INV)
        cloth_extracted = cv2.bitwise_and(cloth_img, cloth_img, mask=mask)

        # Resize dress to fit user's body
        user_height, user_width = user_img.shape[:2]
        dress_resized = cv2.resize(cloth_extracted, (int(user_width * 0.6), int(user_height * 0.6)))

        # Detect face to position dress
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        gray_user = cv2.cvtColor(user_img, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray_user, 1.1, 4)

        # Make a copy of user image for modifications
        result_img = user_img.copy()

        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            face_center_x = x + w // 2
            face_bottom_y = y + h

            # Position dress below face
            x_offset = max(0, min(face_center_x - dress_resized.shape[1] // 2, user_width - dress_resized.shape[1]))
            y_offset = min(face_bottom_y + 10, user_height - dress_resized.shape[0])

            # Overlay dress onto user image
            for y in range(dress_resized.shape[0]):
                if y_offset + y >= user_height:
                    break
                for x in range(dress_resized.shape[1]):
                    if x_offset + x >= user_width:
                        continue
                    # Alpha blending
                    if np.any(dress_resized[y, x] > [20, 20, 20]):  # Non-black pixels
                        result_img[y_offset + y, x_offset + x] = dress_resized[y, x]
        else:
            # If no face detected, place in center
            x_offset = max(0, (user_width - dress_resized.shape[1]) // 2)
            y_offset = max(0, (user_height - dress_resized.shape[0]) // 2)

            # Overlay dress
            for y in range(dress_resized.shape[0]):
                if y_offset + y >= user_height:
                    break
                for x in range(dress_resized.shape[1]):
                    if x_offset + x >= user_width:
                        continue
                    # Alpha blending
                    if np.any(dress_resized[y, x] > [20, 20, 20]):  # Non-black pixels
                        result_img[y_offset + y, x_offset + x] = dress_resized[y, x]

        return Image.fromarray(result_img)
    except Exception as e:
        logger.error(f"Error in virtual try-on: {e}")
        return None

# Handle feedback submission
def submit_user_feedback(rating, text):
    """Process user feedback"""
    logger.info(f"Received feedback - Rating: {rating}, Text: {text}")
    return f"Thank you for your feedback! You rated us {rating}/5."

# HTML for welcome screen
welcome_html = """
<!DOCTYPE html>
<html>
<head>
<style>
  body, html {
    margin: 0;
    padding: 0;
    width: 100%;
    height: 100%;
    font-family: 'Arial', sans-serif;
  }

  .scene {
    width: 100%;
    height: 100vh;
    background: linear-gradient(135deg, #000428, #004e92);
    display: flex;
    justify-content: center;
    align-items: center;
  }

  .card-container {
    width: 60%;
    height: 60%;
    background: orange;
    border-radius: 20px;
    display: flex;
    align-items: center;
    justify-content: center;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.5);
    text-align: center;
  }

  .title {
    font-size: 3em;
    font-weight: bold;
    color: white;
  }

  .subtitle {
    font-size: 1.5em;
    color: white;
    margin-bottom: 20px;
  }

  .start-button {
    padding: 15px 40px;
    background: linear-gradient(135deg, #FFA500, #FF7300);
    color: white;
    font-size: 1.9em;
    border: none;
    border-radius: 50px;
    cursor: pointer;
    box-shadow: 0 5px 15px rgba(255, 165, 0, 0.3);
    transition: transform 0.2s ease-in-out;
  }

  .start-button:hover {
    transform: scale(1.05);
  }
</style>
</head>
<body>
  <div class="scene">
    <div class="card-container">
      <div>
        <h1 class="title">Fashion AI Generator</h1>
        <p class="subtitle">Create stunning fashion designs with AI!</p>
        <br><br><br><br>
        <button class="start-button" onclick="document.getElementById('startButtonGradio').click();">START</button>
      </div>
    </div>
  </div>
</body>
</html>
"""

# Add a mock image generation option for testing if models fail to load
def generate_mock_image():
    """Generate a mock image for testing when models are not available"""
    img = Image.new('RGB', (512, 512), color=(255, 255, 255))
    # Draw some basic shapes to simulate clothing
    from PIL import ImageDraw
    draw = ImageDraw.Draw(img)
    draw.rectangle((100, 150, 412, 450), fill=(200, 150, 150), outline=(100, 50, 50))
    draw.ellipse((206, 80, 306, 180), fill=(255, 200, 200), outline=(100, 50, 50))
    return img

# Build Gradio UI with Tabs
with gr.Blocks() as demo:
    # Status message for model loading
    model_status = gr.Markdown(
        "✅ Models loaded successfully" if MODELS_LOADED else
        "⚠️ AI models failed to load. Using mock images for demonstration."
    )

    # Start Page
    with gr.Group(visible=True) as welcome_page:
        gr.HTML(welcome_html)
        start_btn = gr.Button("Start", visible=False, elem_id="startButtonGradio")

    # Main Application with Tabs (initially hidden)
    with gr.Group(visible=False) as main_app:
        gr.Markdown("# Fashion AI Application")

        with gr.Tabs() as tabs:
            # Tab 1: Fashion Generator
            with gr.TabItem("Fashion Generator"):
                gr.Markdown("## Generate Custom Fashion Designs")

                input_type = gr.Radio(choices=["Dropdowns", "Manual Text"], label="Select Input Type", value="Dropdowns")

                with gr.Row():
                    with gr.Column(scale=1):
                        category = gr.Dropdown(choices=categories, label="Category", value="Dress")
                        style = gr.Dropdown(choices=styles, label="Style", value="Modern")
                        color_scheme = gr.Dropdown(choices=color_schemes, label="Color Scheme", value="Vibrant")
                        pattern = gr.Dropdown(choices=patterns, label="Pattern", value="Plain")
                        material = gr.Dropdown(choices=materials, label="Material", value="Cotton")
                        fit = gr.Dropdown(choices=fits, label="Fit", value="Regular")
                        occasion = gr.Dropdown(choices=occasions, label="Occasion", value="Casual")
                        season = gr.Dropdown(choices=seasons, label="Season", value="Summer")

                        custom_prompt = gr.Textbox(
                            label="Custom Description",
                            placeholder="Describe your fashion design...",
                            interactive=True
                        )

                        with gr.Row():
                            generate_btn = gr.Button("Generate Fashion Image", variant="primary")
                            mock_btn = gr.Button("Generate Mock Image (Test)", visible=not MODELS_LOADED)

                    with gr.Column(scale=1):
                        output_image = gr.Image(label="Generated Fashion Design", type="pil")

                # Feedback Section
                with gr.Group():
                    gr.Markdown("### We'd Love Your Feedback!")
                    rating = gr.Slider(minimum=1, maximum=5, step=1, label="Rate (1-5)", value=5)
                    feedback_text = gr.Textbox(
                        label="Your Feedback",
                        lines=3,
                        placeholder="Your thoughts on the generated design"
                    )
                    submit_feedback = gr.Button("Submit Feedback")
                    feedback_message = gr.Markdown(visible=True)

            # Tab 2: Similar Image Generator
            with gr.TabItem("Similar Image Generator"):
                gr.Markdown("## Generate Similar Fashion Images")

                with gr.Row():
                    with gr.Column(scale=1):
                        reference_image = gr.Image(label="Upload Reference Image", type="numpy")
                        similarity_strength = gr.Slider(
                            minimum=0.1,
                            maximum=0.9,
                            step=0.1,
                            label="Similarity Strength (lower = more similar)",
                            value=0.7
                        )
                        with gr.Row():
                            similar_generate_btn = gr.Button("Generate Similar Image", variant="primary")
                            similar_mock_btn = gr.Button("Generate Mock Similar Image", visible=not MODELS_LOADED)

                    with gr.Column(scale=1):
                        similar_output = gr.Image(label="Generated Similar Image", type="pil")

            # Tab 3: Virtual Try-On
            with gr.TabItem("Virtual Try-On"):
                gr.Markdown("## Virtual Try-On System")

                with gr.Row():
                    with gr.Column(scale=1):
                        user_image = gr.Image(label="Upload Your Photo", type="numpy")
                        cloth_image = gr.Image(label="Upload Clothing Item", type="numpy")
                        tryon_btn = gr.Button("Generate Try-On", variant="primary")

                    with gr.Column(scale=1):
                        tryon_output = gr.Image(label="Virtual Try-On Result", type="pil")

    # Event Handlers
    # Welcome page to main app transition
    start_btn.click(
        lambda: [gr.update(visible=False), gr.update(visible=True)],
        outputs=[welcome_page, main_app]
    )

    # Tab 1: Fashion Generator
    generate_btn.click(
        get_completion,
        inputs=[
            input_type, category, style, color_scheme, pattern,
            material, fit, occasion, season, custom_prompt
        ],
        outputs=output_image
    )

    # Mock button for testing
    if not MODELS_LOADED:
        mock_btn.click(
            generate_mock_image,
            inputs=[],
            outputs=output_image
        )

    submit_feedback.click(
        submit_user_feedback,
        inputs=[rating, feedback_text],
        outputs=feedback_message
    )

    # Tab 2: Similar Image Generator
    similar_generate_btn.click(
        generate_similar_image,
        inputs=[reference_image, similarity_strength],
        outputs=similar_output
    )

    # Mock button for similar image
    if not MODELS_LOADED:
        similar_mock_btn.click(
            generate_mock_image,
            inputs=[],
            outputs=similar_output
        )

    # Tab 3: Virtual Try-On
    tryon_btn.click(
        virtual_try_on,
        inputs=[user_image, cloth_image],
        outputs=tryon_output
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.48.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://943b87998313755569.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [21]:
import torch
import numpy as np
import cv2
import gradio as gr
from PIL import Image
import io
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")

# Load models
try:
    from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline

    # Text-to-image pipeline for Fashion Generation with lower precision for memory efficiency
    txt2img_pipeline = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)

    # Enable attention slicing for lower memory usage
    txt2img_pipeline.enable_attention_slicing()

    # Image-to-image pipeline for Similar Image Generation
    img2img_pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)

    img2img_pipeline.enable_attention_slicing()

    MODELS_LOADED = True
    logger.info("Models loaded successfully")

except Exception as e:
    logger.error(f"Error loading models: {e}")
    MODELS_LOADED = False
    # Create dummy pipelines for demo purposes if loading fails
    class DummyPipeline:
        def __call__(self, **kwargs):
            class DummyOutput:
                images = [Image.new('RGB', (512, 512), color=(200, 200, 200))]
            return DummyOutput()
    txt2img_pipeline = DummyPipeline()
    img2img_pipeline = DummyPipeline()

# Define categories and attributes for Fashion Generator
categories = ["Dress", "T-shirt", "Jeans", "Jacket", "Shirt and Jeans", "Lehenga", "Churidar", "Saree", "Gown"]
styles = ["Modern", "Vintage", "Casual", "Elegant", "Traditional"]
color_schemes = ["Monochrome", "Pastel", "Vibrant"]
patterns = ["Striped", "Floral", "Plain", "Embroidery"]
materials = ["Cotton", "Silk", "Denim", "Leather", "Georgette", "Crepe"]
fits = ["Slim", "Regular", "Loose"]
occasions = ["Formal", "Casual", "Party"]
seasons = ["Summer", "Winter", "Normal"]

# Function for Fashion Generation
def get_completion(input_type, category, style, color_scheme, pattern, material, fit, occasion, season, custom_prompt):
    """Generate fashion image based on selected parameters or custom prompt"""
    try:
        if not MODELS_LOADED:
            return Image.new('RGB', (512, 512), color=(200, 200, 200))

        if input_type == "Dropdowns":
            prompt = f"High quality fashion photography of a {fit} {style} {category} made of {material} with {pattern} pattern in {color_scheme} colors, suitable for {occasion} wear during {season} season"
        else:
            prompt = custom_prompt if custom_prompt else "A fashion item"

        logger.info(f"Generating image with prompt: {prompt}")

        # Use lower number of inference steps for faster generation
        image = txt2img_pipeline(
            prompt=prompt,
            num_inference_steps=25,
            guidance_scale=7.5
        ).images[0]

        return image
    except Exception as e:
        logger.error(f"Error in fashion generation: {e}")
        # Return a red image with error message
        error_img = Image.new('RGB', (512, 512), color=(255, 200, 200))
        return error_img

# Function for Similar Image Generation
def generate_similar_image(reference_image, strength=0.7):
    """Generate an image similar to the uploaded reference image using img2img"""
    try:
        if reference_image is None or not MODELS_LOADED:
            return Image.new('RGB', (512, 512), color=(200, 200, 200))

        if isinstance(reference_image, np.ndarray):
            reference_image = Image.fromarray(reference_image)

        # Ensure image is RGB
        reference_image = reference_image.convert("RGB").resize((512, 512))

        logger.info(f"Generating similar image with strength: {strength}")

        similar_image = img2img_pipeline(
            prompt="high quality fashion item",
            image=reference_image,
            strength=strength,
            guidance_scale=7.5,
            num_inference_steps=25
        ).images[0]

        return similar_image
    except Exception as e:
        logger.error(f"Error in similar image generation: {e}")
        # Return image with error
        error_img = Image.new('RGB', (512, 512), color=(255, 200, 200))
        return error_img

# Function to overlay face onto model (unchanged)
def overlay_face(base_image, face_image):
    """Overlay the face image onto the base image (placeholder implementation)"""
    try:
        if face_image is None or base_image is None:
            return base_image

        # Convert images to numpy arrays if they're not already
        if isinstance(base_image, Image.Image):
            base_np = np.array(base_image)
        else:
            base_np = base_image

        if isinstance(face_image, Image.Image):
            face_np = np.array(face_image)
        else:
            face_np = face_image

        # Convert to RGB if needed
        if len(base_np.shape) == 2 or base_np.shape[2] == 1:
            base_np = cv2.cvtColor(base_np, cv2.COLOR_GRAY2RGB)
        if len(face_np.shape) == 2 or face_np.shape[2] == 1:
            face_np = cv2.cvtColor(face_np, cv2.COLOR_GRAY2RGB)

        # Detect face in the face image
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        gray_face = cv2.cvtColor(face_np, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray_face, 1.1, 4)

        if len(faces) == 0:
            return Image.fromarray(base_np)

        (fx, fy, fw, fh) = faces[0]
        extracted_face = face_np[fy:fy+fh, fx:fx+fw]

        # Detect face in base image
        gray_base = cv2.cvtColor(base_np, cv2.COLOR_RGB2GRAY)
        base_faces = face_cascade.detectMultiScale(gray_base, 1.1, 4)

        if len(base_faces) == 0:
            # If no face detected in base, place at a reasonable position
            bx, by = base_np.shape[1] // 2 - fw // 2, base_np.shape[0] // 4 - fh // 2
        else:
            (bx, by, bw, bh) = base_faces[0]
            # Resize extracted face to match base face size
            extracted_face = cv2.resize(extracted_face, (bw, bh))
            fw, fh = bw, bh

        # Create mask for blending
        mask = np.zeros((fh, fw), dtype=np.uint8)
        mask = cv2.ellipse(mask, (fw//2, fh//2), (fw//2-5, fh//2-5), 0, 0, 360, 255, -1)
        mask = cv2.GaussianBlur(mask, (11, 11), 0)

        # Apply mask and blend faces
        for c in range(3):
            base_np[by:by+fh, bx:bx+fw, c] = (
                base_np[by:by+fh, bx:bx+fw, c] * (255 - mask) / 255 +
                extracted_face[:, :, c] * mask / 255
            )

        return Image.fromarray(base_np)
    except Exception as e:
        logger.error(f"Error in face overlay: {e}")
        return base_image if isinstance(base_image, Image.Image) else Image.fromarray(base_image)

# Improved function for Virtual Try-On
def virtual_try_on(user_image, cloth_image):
    """Overlay cloth image onto user image with improved positioning and blending"""
    try:
        if user_image is None or cloth_image is None:
            return None

        # Convert to numpy arrays if needed
        if isinstance(user_image, Image.Image):
            user_img = np.array(user_image)
        else:
            user_img = user_image

        if isinstance(cloth_image, Image.Image):
            cloth_img = np.array(cloth_image)
        else:
            cloth_img = cloth_image

        # Convert to RGB if needed
        if len(user_img.shape) == 2 or user_img.shape[2] == 1:
            user_img = cv2.cvtColor(user_img, cv2.COLOR_GRAY2RGB)
        if len(cloth_img.shape) == 2 or cloth_img.shape[2] == 1:
            cloth_img = cv2.cvtColor(cloth_img, cv2.COLOR_GRAY2RGB)

        # Step 1: Extract the clothing item from its background
        gray_cloth = cv2.cvtColor(cloth_img, cv2.COLOR_RGB2GRAY)
        _, mask = cv2.threshold(gray_cloth, 200, 255, cv2.THRESH_BINARY_INV)  # Create a binary mask

        # Apply morphological operations to improve mask
        kernel = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        cloth_extracted = cv2.bitwise_and(cloth_img, cloth_img, mask=mask)

        # Step 2: Resize the clothing to fit the user's body
        user_height, user_width = user_img.shape[:2]
        cloth_height, cloth_width = cloth_extracted.shape[:2]

        # Calculate appropriate size based on user image dimensions
        target_width = int(user_width * 0.6)
        target_height = int(cloth_height * (target_width / cloth_width))

        # Ensure the clothing isn't too tall for the image
        if target_height > user_height * 0.8:
            target_height = int(user_height * 0.8)
            target_width = int(cloth_width * (target_height / cloth_height))

        cloth_resized = cv2.resize(cloth_extracted, (target_width, target_height))
        mask_resized = cv2.resize(mask, (target_width, target_height))

        # Step 3: Detect face to position clothing correctly
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        gray_user = cv2.cvtColor(user_img, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray_user, 1.1, 4)

        # Make a copy of user image for modifications
        result_img = user_img.copy()

        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            face_center_x = x + w // 2
            face_bottom_y = y + h

            # Calculate shoulder position (approximate based on face)
            shoulder_y = face_bottom_y + int(h * 0.5)

            # Position clothing below face with proper centering
            x_offset = max(0, min(face_center_x - cloth_resized.shape[1] // 2,
                                  user_width - cloth_resized.shape[1]))
            y_offset = max(0, min(shoulder_y, user_height - cloth_resized.shape[0]))
        else:
            # If no face detected, place in center of top third
            x_offset = max(0, (user_width - cloth_resized.shape[1]) // 2)
            y_offset = max(0, (user_height // 3) - (cloth_resized.shape[0] // 4))

        # Step 4: Create a proper alpha mask for blending
        alpha_mask = mask_resized.astype(float) / 255.0

        # Step 5: Blend the clothing onto the user image
        for c in range(3):
            for y in range(cloth_resized.shape[0]):
                if y_offset + y >= user_height:
                    break
                for x in range(cloth_resized.shape[1]):
                    if x_offset + x >= user_width:
                        continue
                    # Only blend where mask is non-zero
                    if alpha_mask[y, x] > 0.1:
                        # Blend clothing with user image
                        alpha = alpha_mask[y, x] * 0.8  # Adjustable opacity
                        result_img[y_offset + y, x_offset + x, c] = (
                            (1 - alpha) * result_img[y_offset + y, x_offset + x, c] +
                            alpha * cloth_resized[y, x, c]
                        )

        return Image.fromarray(result_img)
    except Exception as e:
        logger.error(f"Error in virtual try-on: {e}")
        return user_image if isinstance(user_image, Image.Image) else Image.fromarray(user_image)

# Handle feedback submission
def submit_user_feedback(rating, text):
    """Process user feedback"""
    logger.info(f"Received feedback - Rating: {rating}, Text: {text}")
    return f"Thank you for your feedback! You rated us {rating}/5."

# HTML for welcome screen
welcome_html = """
<!DOCTYPE html>
<html>
<head>
<style>
  body, html {
    margin: 0;
    padding: 0;
    width: 100%;
    height: 100%;
    font-family: 'Arial', sans-serif;
  }

  .scene {
    width: 100%;
    height: 100vh;
    background: linear-gradient(135deg, #000428, #004e92);
    display: flex;
    justify-content: center;
    align-items: center;
  }

  .card-container {
    width: 60%;
    height: 60%;
    background: orange;
    border-radius: 20px;
    display: flex;
    align-items: center;
    justify-content: center;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.5);
    text-align: center;
  }

  .title {
    font-size: 3em;
    font-weight: bold;
    color: white;
  }

  .subtitle {
    font-size: 1.5em;
    color: white;
    margin-bottom: 20px;
  }

  .start-button {
    padding: 15px 40px;
    background: linear-gradient(135deg, #FFA500, #FF7300);
    color: white;
    font-size: 1.9em;
    border: none;
    border-radius: 50px;
    cursor: pointer;
    box-shadow: 0 5px 15px rgba(255, 165, 0, 0.3);
    transition: transform 0.2s ease-in-out;
  }

  .start-button:hover {
    transform: scale(1.05);
  }
</style>
</head>
<body>
  <div class="scene">
    <div class="card-container">
      <div>
        <h1 class="title">Fashion AI Generator</h1>
        <p class="subtitle">Create stunning fashion designs with AI!</p>
        <br><br><br><br>
        <button class="start-button" onclick="document.getElementById('startButtonGradio').click();">START</button>
      </div>
    </div>
  </div>
</body>
</html>
"""

# Add a mock image generation option for testing if models fail to load
def generate_mock_image():
    """Generate a mock image for testing when models are not available"""
    img = Image.new('RGB', (512, 512), color=(255, 255, 255))
    # Draw some basic shapes to simulate clothing
    from PIL import ImageDraw
    draw = ImageDraw.Draw(img)
    draw.rectangle((100, 150, 412, 450), fill=(200, 150, 150), outline=(100, 50, 50))
    draw.ellipse((206, 80, 306, 180), fill=(255, 200, 200), outline=(100, 50, 50))
    return img

# Build Gradio UI with Tabs
with gr.Blocks() as demo:
    # Status message for model loading
    model_status = gr.Markdown(
        "✅ Models loaded successfully" if MODELS_LOADED else
        "⚠️ AI models failed to load. Using mock images for demonstration."
    )

    # Start Page
    with gr.Group(visible=True) as welcome_page:
        gr.HTML(welcome_html)
        start_btn = gr.Button("Start", visible=False, elem_id="startButtonGradio")

    # Main Application with Tabs (initially hidden)
    with gr.Group(visible=False) as main_app:
        gr.Markdown("# Fashion AI Application")

        with gr.Tabs() as tabs:
            # Tab 1: Fashion Generator
            with gr.TabItem("Fashion Generator"):
                gr.Markdown("## Generate Custom Fashion Designs")

                input_type = gr.Radio(choices=["Dropdowns", "Manual Text"], label="Select Input Type", value="Dropdowns")

                with gr.Row():
                    with gr.Column(scale=1):
                        category = gr.Dropdown(choices=categories, label="Category", value="Dress")
                        style = gr.Dropdown(choices=styles, label="Style", value="Modern")
                        color_scheme = gr.Dropdown(choices=color_schemes, label="Color Scheme", value="Vibrant")
                        pattern = gr.Dropdown(choices=patterns, label="Pattern", value="Plain")
                        material = gr.Dropdown(choices=materials, label="Material", value="Cotton")
                        fit = gr.Dropdown(choices=fits, label="Fit", value="Regular")
                        occasion = gr.Dropdown(choices=occasions, label="Occasion", value="Casual")
                        season = gr.Dropdown(choices=seasons, label="Season", value="Summer")

                        custom_prompt = gr.Textbox(
                            label="Custom Description",
                            placeholder="Describe your fashion design...",
                            interactive=True
                        )

                        with gr.Row():
                            generate_btn = gr.Button("Generate Fashion Image", variant="primary")
                            mock_btn = gr.Button("Generate Mock Image (Test)", visible=not MODELS_LOADED)

                    with gr.Column(scale=1):
                        output_image = gr.Image(label="Generated Fashion Design", type="pil")

                # Feedback Section
                with gr.Group():
                    gr.Markdown("### We'd Love Your Feedback!")
                    rating = gr.Slider(minimum=1, maximum=5, step=1, label="Rate (1-5)", value=5)
                    feedback_text = gr.Textbox(
                        label="Your Feedback",
                        lines=3,
                        placeholder="Your thoughts on the generated design"
                    )
                    submit_feedback = gr.Button("Submit Feedback")
                    feedback_message = gr.Markdown(visible=True)

            # Tab 2: Similar Image Generator
            with gr.TabItem("Similar Image Generator"):
                gr.Markdown("## Generate Similar Fashion Images")

                with gr.Row():
                    with gr.Column(scale=1):
                        reference_image = gr.Image(label="Upload Reference Image", type="numpy")
                        similarity_strength = gr.Slider(
                            minimum=0.1,
                            maximum=0.9,
                            step=0.1,
                            label="Similarity Strength (lower = more similar)",
                            value=0.7
                        )
                        with gr.Row():
                            similar_generate_btn = gr.Button("Generate Similar Image", variant="primary")
                            similar_mock_btn = gr.Button("Generate Mock Similar Image", visible=not MODELS_LOADED)

                    with gr.Column(scale=1):
                        similar_output = gr.Image(label="Generated Similar Image", type="pil")

            # Tab 3: Virtual Try-On
            with gr.TabItem("Virtual Try-On"):
                gr.Markdown("## Virtual Try-On System")

                with gr.Row():
                    with gr.Column(scale=1):
                        user_image = gr.Image(label="Upload Your Photo", type="numpy")
                        cloth_image = gr.Image(label="Upload Clothing Item", type="numpy")
                        tryon_btn = gr.Button("Generate Try-On", variant="primary")

                    with gr.Column(scale=1):
                        tryon_output = gr.Image(label="Virtual Try-On Result", type="pil")

    # Event Handlers
    # Welcome page to main app transition
    start_btn.click(
        lambda: [gr.update(visible=False), gr.update(visible=True)],
        outputs=[welcome_page, main_app]
    )

    # Tab 1: Fashion Generator
    generate_btn.click(
        get_completion,
        inputs=[
            input_type, category, style, color_scheme, pattern,
            material, fit, occasion, season, custom_prompt
        ],
        outputs=output_image
    )

    # Mock button for testing
    if not MODELS_LOADED:
        mock_btn.click(
            generate_mock_image,
            inputs=[],
            outputs=output_image
        )

    submit_feedback.click(
        submit_user_feedback,
        inputs=[rating, feedback_text],
        outputs=feedback_message
    )

    # Tab 2: Similar Image Generator
    similar_generate_btn.click(
        generate_similar_image,
        inputs=[reference_image, similarity_strength],
        outputs=similar_output
    )

    # Mock button for similar image
    if not MODELS_LOADED:
        similar_mock_btn.click(
            generate_mock_image,
            inputs=[],
            outputs=similar_output
        )

    # Tab 3: Virtual Try-On
    tryon_btn.click(
        virtual_try_on,
        inputs=[user_image, cloth_image],
        outputs=tryon_output
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.48.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://5e1eb5546501599454.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [20]:
import torch
import numpy as np
import cv2
import gradio as gr
from PIL import Image
import io
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")

# Load models
try:
    from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline

    # Text-to-image pipeline for Fashion Generation with lower precision for memory efficiency
    txt2img_pipeline = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)

    # Enable attention slicing for lower memory usage
    txt2img_pipeline.enable_attention_slicing()

    # Image-to-image pipeline for Similar Image Generation
    img2img_pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)

    img2img_pipeline.enable_attention_slicing()

    MODELS_LOADED = True
    logger.info("Models loaded successfully")

except Exception as e:
    logger.error(f"Error loading models: {e}")
    MODELS_LOADED = False
    # Create dummy pipelines for demo purposes if loading fails
    class DummyPipeline:
        def __call__(self, **kwargs):
            class DummyOutput:
                images = [Image.new('RGB', (512, 512), color=(200, 200, 200))]
            return DummyOutput()
    txt2img_pipeline = DummyPipeline()
    img2img_pipeline = DummyPipeline()

# Define categories and attributes for Fashion Generator
categories = ["Dress", "T-shirt", "Jeans", "Jacket", "Shirt and Jeans", "Lehenga", "Churidar", "Saree", "Gown"]
styles = ["Modern", "Vintage", "Casual", "Elegant", "Traditional"]
color_schemes = ["Monochrome", "Pastel", "Vibrant"]
patterns = ["Striped", "Floral", "Plain", "Embroidery"]
materials = ["Cotton", "Silk", "Denim", "Leather", "Georgette", "Crepe"]
fits = ["Slim", "Regular", "Loose"]
occasions = ["Formal", "Casual", "Party"]
seasons = ["Summer", "Winter", "Normal"]

# Function for Fashion Generation
def get_completion(input_type, category, style, color_scheme, pattern, material, fit, occasion, season, custom_prompt):
    """Generate fashion image based on selected parameters or custom prompt"""
    try:
        if not MODELS_LOADED:
            return Image.new('RGB', (512, 512), color=(200, 200, 200))

        if input_type == "Dropdowns":
            prompt = f"High quality fashion photography of a {fit} {style} {category} made of {material} with {pattern} pattern in {color_scheme} colors, suitable for {occasion} wear during {season} season"
        else:
            prompt = custom_prompt if custom_prompt else "A fashion item"

        logger.info(f"Generating image with prompt: {prompt}")

        # Use lower number of inference steps for faster generation
        image = txt2img_pipeline(
            prompt=prompt,
            num_inference_steps=25,
            guidance_scale=7.5
        ).images[0]

        return image
    except Exception as e:
        logger.error(f"Error in fashion generation: {e}")
        # Return a red image with error message
        error_img = Image.new('RGB', (512, 512), color=(255, 200, 200))
        return error_img

# Function for Similar Image Generation
def generate_similar_image(reference_image, strength=0.7):
    """Generate an image similar to the uploaded reference image using img2img"""
    try:
        if reference_image is None or not MODELS_LOADED:
            return Image.new('RGB', (512, 512), color=(200, 200, 200))

        if isinstance(reference_image, np.ndarray):
            reference_image = Image.fromarray(reference_image)

        # Ensure image is RGB
        reference_image = reference_image.convert("RGB").resize((512, 512))

        logger.info(f"Generating similar image with strength: {strength}")

        similar_image = img2img_pipeline(
            prompt="high quality fashion item",
            image=reference_image,
            strength=strength,
            guidance_scale=7.5,
            num_inference_steps=25
        ).images[0]

        return similar_image
    except Exception as e:
        logger.error(f"Error in similar image generation: {e}")
        # Return image with error
        error_img = Image.new('RGB', (512, 512), color=(255, 200, 200))
        return error_img

# Function to overlay face onto model (unchanged)
def overlay_face(base_image, face_image):
    """Overlay the face image onto the base image (placeholder implementation)"""
    try:
        if face_image is None or base_image is None:
            return base_image

        # Convert images to numpy arrays if they're not already
        if isinstance(base_image, Image.Image):
            base_np = np.array(base_image)
        else:
            base_np = base_image

        if isinstance(face_image, Image.Image):
            face_np = np.array(face_image)
        else:
            face_np = face_image

        # Convert to RGB if needed
        if len(base_np.shape) == 2 or base_np.shape[2] == 1:
            base_np = cv2.cvtColor(base_np, cv2.COLOR_GRAY2RGB)
        if len(face_np.shape) == 2 or face_np.shape[2] == 1:
            face_np = cv2.cvtColor(face_np, cv2.COLOR_GRAY2RGB)

        # Detect face in the face image
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        gray_face = cv2.cvtColor(face_np, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray_face, 1.1, 4)

        if len(faces) == 0:
            return Image.fromarray(base_np)

        (fx, fy, fw, fh) = faces[0]
        extracted_face = face_np[fy:fy+fh, fx:fx+fw]

        # Detect face in base image
        gray_base = cv2.cvtColor(base_np, cv2.COLOR_RGB2GRAY)
        base_faces = face_cascade.detectMultiScale(gray_base, 1.1, 4)

        if len(base_faces) == 0:
            # If no face detected in base, place at a reasonable position
            bx, by = base_np.shape[1] // 2 - fw // 2, base_np.shape[0] // 4 - fh // 2
        else:
            (bx, by, bw, bh) = base_faces[0]
            # Resize extracted face to match base face size
            extracted_face = cv2.resize(extracted_face, (bw, bh))
            fw, fh = bw, bh

        # Create mask for blending
        mask = np.zeros((fh, fw), dtype=np.uint8)
        mask = cv2.ellipse(mask, (fw//2, fh//2), (fw//2-5, fh//2-5), 0, 0, 360, 255, -1)
        mask = cv2.GaussianBlur(mask, (11, 11), 0)

        # Apply mask and blend faces
        for c in range(3):
            base_np[by:by+fh, bx:bx+fw, c] = (
                base_np[by:by+fh, bx:bx+fw, c] * (255 - mask) / 255 +
                extracted_face[:, :, c] * mask / 255
            )

        return Image.fromarray(base_np)
    except Exception as e:
        logger.error(f"Error in face overlay: {e}")
        return base_image if isinstance(base_image, Image.Image) else Image.fromarray(base_image)

# Improved function for Virtual Try-On
def virtual_try_on(user_image, cloth_image):
    """Overlay cloth image onto user image with improved positioning and blending"""
    try:
        if user_image is None or cloth_image is None:
            return None

        # Convert to numpy arrays if needed
        if isinstance(user_image, Image.Image):
            user_img = np.array(user_image)
        else:
            user_img = user_image

        if isinstance(cloth_image, Image.Image):
            cloth_img = np.array(cloth_image)
        else:
            cloth_img = cloth_image

        # Convert to RGB if needed
        if len(user_img.shape) == 2 or user_img.shape[2] == 1:
            user_img = cv2.cvtColor(user_img, cv2.COLOR_GRAY2RGB)
        if len(cloth_img.shape) == 2 or cloth_img.shape[2] == 1:
            cloth_img = cv2.cvtColor(cloth_img, cv2.COLOR_GRAY2RGB)

        # Step 1: Extract the clothing item from its background
        gray_cloth = cv2.cvtColor(cloth_img, cv2.COLOR_RGB2GRAY)
        _, mask = cv2.threshold(gray_cloth, 200, 255, cv2.THRESH_BINARY_INV)  # Create a binary mask

        # Apply morphological operations to improve mask
        kernel = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        cloth_extracted = cv2.bitwise_and(cloth_img, cloth_img, mask=mask)

        # Step 2: Resize the clothing to fit the user's body
        user_height, user_width = user_img.shape[:2]
        cloth_height, cloth_width = cloth_extracted.shape[:2]

        # Calculate appropriate size based on user image dimensions
        target_width = int(user_width * 0.6)
        target_height = int(cloth_height * (target_width / cloth_width))

        # Ensure the clothing isn't too tall for the image
        if target_height > user_height * 0.8:
            target_height = int(user_height * 0.8)
            target_width = int(cloth_width * (target_height / cloth_height))

        cloth_resized = cv2.resize(cloth_extracted, (target_width, target_height))
        mask_resized = cv2.resize(mask, (target_width, target_height))

        # Step 3: Detect face to position clothing correctly
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        gray_user = cv2.cvtColor(user_img, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray_user, 1.1, 4)

        # Make a copy of user image for modifications
        result_img = user_img.copy()

        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            face_center_x = x + w // 2
            face_bottom_y = y + h

            # Calculate shoulder position (approximate based on face)
            shoulder_y = face_bottom_y + int(h * 0.5)

            # Position clothing below face with proper centering
            x_offset = max(0, min(face_center_x - cloth_resized.shape[1] // 2,
                                  user_width - cloth_resized.shape[1]))
            y_offset = max(0, min(shoulder_y, user_height - cloth_resized.shape[0]))
        else:
            # If no face detected, place in center of top third
            x_offset = max(0, (user_width - cloth_resized.shape[1]) // 2)
            y_offset = max(0, (user_height // 3) - (cloth_resized.shape[0] // 4))

        # Step 4: Create a proper alpha mask for blending
        alpha_mask = mask_resized.astype(float) / 255.0

        # Step 5: Blend the clothing onto the user image
        for c in range(3):
            for y in range(cloth_resized.shape[0]):
                if y_offset + y >= user_height:
                    break
                for x in range(cloth_resized.shape[1]):
                    if x_offset + x >= user_width:
                        continue
                    # Only blend where mask is non-zero
                    if alpha_mask[y, x] > 0.1:
                        # Blend clothing with user image
                        alpha = alpha_mask[y, x] * 0.8  # Adjustable opacity
                        result_img[y_offset + y, x_offset + x, c] = (
                            (1 - alpha) * result_img[y_offset + y, x_offset + x, c] +
                            alpha * cloth_resized[y, x, c]
                        )

        return Image.fromarray(result_img)
    except Exception as e:
        logger.error(f"Error in virtual try-on: {e}")
        return user_image if isinstance(user_image, Image.Image) else Image.fromarray(user_image)

# Handle feedback submission
def submit_user_feedback(rating, text):
    """Process user feedback"""
    logger.info(f"Received feedback - Rating: {rating}, Text: {text}")
    return f"Thank you for your feedback! You rated us {rating}/5."

# HTML for welcome screen
welcome_html = """
<!DOCTYPE html>
<html>
<head>
<style>
  body, html {
    margin: 0;
    padding: 0;
    width: 100%;
    height: 100%;
    font-family: 'Arial', sans-serif;
  }

  .scene {
    width: 100%;
    height: 100vh;
    background: linear-gradient(135deg, #000428, #004e92);
    display: flex;
    justify-content: center;
    align-items: center;
  }

  .card-container {
    width: 60%;
    height: 60%;
    background: orange;
    border-radius: 20px;
    display: flex;
    align-items: center;
    justify-content: center;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.5);
    text-align: center;
  }

  .title {
    font-size: 3em;
    font-weight: bold;
    color: white;
  }

  .subtitle {
    font-size: 1.5em;
    color: white;
    margin-bottom: 20px;
  }

  .start-button {
    padding: 15px 40px;
    background: linear-gradient(135deg, #FFA500, #FF7300);
    color: white;
    font-size: 1.9em;
    border: none;
    border-radius: 50px;
    cursor: pointer;
    box-shadow: 0 5px 15px rgba(255, 165, 0, 0.3);
    transition: transform 0.2s ease-in-out;
  }

  .start-button:hover {
    transform: scale(1.05);
  }
</style>
</head>
<body>
  <div class="scene">
    <div class="card-container">
      <div>
        <h1 class="title">Fashion AI Generator</h1>
        <p class="subtitle">Create stunning fashion designs with AI!</p>
        <br><br><br><br>
        <button class="start-button" onclick="document.getElementById('startButtonGradio').click();">START</button>
      </div>
    </div>
  </div>
</body>
</html>
"""

# Add a mock image generation option for testing if models fail to load
def generate_mock_image():
    """Generate a mock image for testing when models are not available"""
    img = Image.new('RGB', (512, 512), color=(255, 255, 255))
    # Draw some basic shapes to simulate clothing
    from PIL import ImageDraw
    draw = ImageDraw.Draw(img)
    draw.rectangle((100, 150, 412, 450), fill=(200, 150, 150), outline=(100, 50, 50))
    draw.ellipse((206, 80, 306, 180), fill=(255, 200, 200), outline=(100, 50, 50))
    return img

# Build Gradio UI with Tabs
with gr.Blocks() as demo:
    # Status message for model loading
    model_status = gr.Markdown(
        "✅ Models loaded successfully" if MODELS_LOADED else
        "⚠️ AI models failed to load. Using mock images for demonstration."
    )

    # Start Page
    with gr.Group(visible=True) as welcome_page:
        gr.HTML(welcome_html)
        start_btn = gr.Button("Start", visible=False, elem_id="startButtonGradio")

    # Main Application with Tabs (initially hidden)
    with gr.Group(visible=False) as main_app:
        gr.Markdown("# Fashion AI Application")

        with gr.Tabs() as tabs:
            # Tab 1: Fashion Generator
            with gr.TabItem("Fashion Generator"):
                gr.Markdown("## Generate Custom Fashion Designs")

                input_type = gr.Radio(choices=["Dropdowns", "Manual Text"], label="Select Input Type", value="Dropdowns")

                with gr.Row():
                    with gr.Column(scale=1):
                        category = gr.Dropdown(choices=categories, label="Category", value="Dress")
                        style = gr.Dropdown(choices=styles, label="Style", value="Modern")
                        color_scheme = gr.Dropdown(choices=color_schemes, label="Color Scheme", value="Vibrant")
                        pattern = gr.Dropdown(choices=patterns, label="Pattern", value="Plain")
                        material = gr.Dropdown(choices=materials, label="Material", value="Cotton")
                        fit = gr.Dropdown(choices=fits, label="Fit", value="Regular")
                        occasion = gr.Dropdown(choices=occasions, label="Occasion", value="Casual")
                        season = gr.Dropdown(choices=seasons, label="Season", value="Summer")

                        custom_prompt = gr.Textbox(
                            label="Custom Description",
                            placeholder="Describe your fashion design...",
                            interactive=True
                        )

                        with gr.Row():
                            generate_btn = gr.Button("Generate Fashion Image", variant="primary")
                            mock_btn = gr.Button("Generate Mock Image (Test)", visible=not MODELS_LOADED)

                    with gr.Column(scale=1):
                        output_image = gr.Image(label="Generated Fashion Design", type="pil")

                # Feedback Section
                with gr.Group():
                    gr.Markdown("### We'd Love Your Feedback!")
                    rating = gr.Slider(minimum=1, maximum=5, step=1, label="Rate (1-5)", value=5)
                    feedback_text = gr.Textbox(
                        label="Your Feedback",
                        lines=3,
                        placeholder="Your thoughts on the generated design"
                    )
                    submit_feedback = gr.Button("Submit Feedback")
                    feedback_message = gr.Markdown(visible=True)

            # Tab 2: Similar Image Generator
            with gr.TabItem("Similar Image Generator"):
                gr.Markdown("## Generate Similar Fashion Images")

                with gr.Row():
                    with gr.Column(scale=1):
                        reference_image = gr.Image(label="Upload Reference Image", type="numpy")
                        similarity_strength = gr.Slider(
                            minimum=0.1,
                            maximum=0.9,
                            step=0.1,
                            label="Similarity Strength (lower = more similar)",
                            value=0.7
                        )
                        with gr.Row():
                            similar_generate_btn = gr.Button("Generate Similar Image", variant="primary")
                            similar_mock_btn = gr.Button("Generate Mock Similar Image", visible=not MODELS_LOADED)

                    with gr.Column(scale=1):
                        similar_output = gr.Image(label="Generated Similar Image", type="pil")

            # Tab 3: Virtual Try-On
            with gr.TabItem("Virtual Try-On"):
                gr.Markdown("## Virtual Try-On System")

                with gr.Row():
                    with gr.Column(scale=1):
                        user_image = gr.Image(label="Upload Your Photo", type="numpy")
                        cloth_image = gr.Image(label="Upload Clothing Item", type="numpy")
                        tryon_btn = gr.Button("Generate Try-On", variant="primary")

                    with gr.Column(scale=1):
                        tryon_output = gr.Image(label="Virtual Try-On Result", type="pil")

    # Event Handlers
    # Welcome page to main app transition
    start_btn.click(
        lambda: [gr.update(visible=False), gr.update(visible=True)],
        outputs=[welcome_page, main_app]
    )

    # Tab 1: Fashion Generator
    generate_btn.click(
        get_completion,
        inputs=[
            input_type, category, style, color_scheme, pattern,
            material, fit, occasion, season, custom_prompt
        ],
        outputs=output_image
    )

    # Mock button for testing
    if not MODELS_LOADED:
        mock_btn.click(
            generate_mock_image,
            inputs=[],
            outputs=output_image
        )

    submit_feedback.click(
        submit_user_feedback,
        inputs=[rating, feedback_text],
        outputs=feedback_message
    )

    # Tab 2: Similar Image Generator
    similar_generate_btn.click(
        generate_similar_image,
        inputs=[reference_image, similarity_strength],
        outputs=similar_output
    )

    # Mock button for similar image
    if not MODELS_LOADED:
        similar_mock_btn.click(
            generate_mock_image,
            inputs=[],
            outputs=similar_output
        )

    # Tab 3: Virtual Try-On
    tryon_btn.click(
        virtual_try_on,
        inputs=[user_image, cloth_image],
        outputs=tryon_output
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.48.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://27c01f2f405e6d0b21.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
